In [4]:
# Paths to the directories
dir_predicted = 'Data for SAM comparison/Cystanalyser and SAM Segmentations'
dir_GT = 'Data for SAM comparison/Ground Truth'

import numpy as np
import os
import glob
import tifffile as tif


def compute_iou(y_true, y_pred):
    intersection = np.logical_and(y_true, y_pred)
    union = np.logical_or(y_true, y_pred)
    iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

def check_matching_pairs(image_GT, image_pred):

    #Function to check each matching pair of cysts and store the Jaccard similarity index
    #In the case they overlap more than 50% of the area of the GT cyst

    # Initialize an empty dictionary to store the matching pairs and their Jaccard similarity index
    matching_pairs = {}

    # Iterate over each unique nonzero intensity i in image_GT
    for num,i in enumerate(np.unique(image_GT[image_GT > 0])):


        pixels_i_GT = (image_GT == i)

        pixels_i_pred = image_pred[pixels_i_GT]


        # Initialize the best match and its ratio
        best_match = None
        best_ratio = 0

        # Iterate over each unique nonzero intensity j in the pixels from image_SAM
        for j in np.unique(pixels_i_pred[pixels_i_pred > 0]):

            # Compute the intersection and union of pixels with intensity i in image_GT and pixels with intensity j in image_SAM
            intersection = np.logical_and(pixels_i_GT, image_pred == j)

            # Compute the ratio of the size of the intersection to the size of pixels with intensity i in image_GT
            ratio = np.sum(intersection) / np.sum(pixels_i_GT)
            # If this ratio is greater than 0.5 and better than the current best match, compute the Jaccard similarity index
            if ratio > 0.5 and ratio > best_ratio:
                best_match = j
                best_ratio = ratio

        # If there is a best match, store i, best_match, and the Jaccard similarity index in the dictionary
        if best_match:
            intersection = np.logical_and(pixels_i_GT, image_pred == best_match)
            union = np.logical_or(pixels_i_GT, image_pred == best_match)
            jaccard = intersection.sum() / union.sum()
            matching_pairs[num] = (i,best_match, jaccard)
        else:
            matching_pairs[num] = (i, 0, 0)
    
    return matching_pairs


# Iterate over all .tif files in directory GT
#for tif_file in glob.glob(os.path.join(dir_predicted, '*.tif')):

numCysts_GT = list()
cystProp_GT = list()

ious_SAM = list()
numCysts_SAM = list()
cystProp_SAM = list()
matching_pairs_means_SAM = list()
true_detected_cysts_SAM = list()
false_detected_cysts_SAM = list()   

ious_cystAn = list()
numCysts_cystAn = list()
cystProp_cystAn = list()
matching_pairs_means_cystAn = list()
true_detected_cysts_cystAn = list()
false_detected_cysts_cystAn = list()

for tif_file_GT in glob.glob(os.path.join(dir_GT, '*.tif')):

    image_GT = np.array(tif.imread(tif_file_GT))
    image_GT_bin = 1*(image_GT>0)

    base_name = os.path.basename(tif_file_GT)

    numCysts_GT.append(len(np.unique(image_GT))-1)
    cystProp_GT.append(np.sum(image_GT_bin)/np.product(image_GT_bin.shape))

    print('Testing {}'.format(base_name)) 


    image_SAM_name = os.path.join(dir_predicted, 'cystsMask-'+base_name)
    image_SAM = tif.imread(image_SAM_name)
    image_SAM_bin = 1*(image_SAM>0)
    
    ious_SAM.append(compute_iou(image_GT_bin, image_SAM_bin))
    numCysts_SAM.append(len(np.unique(image_SAM))-1)
    cystProp_SAM.append(np.sum(image_SAM_bin)/np.product(image_SAM_bin.shape))
    matching_pairs_SAM = check_matching_pairs(image_SAM, image_GT)
    matching_pairs_means_SAM.append(np.mean([pair[2] for pair in matching_pairs_SAM.values()]))

    true_det = 0
    false_det = 0
    for pair in matching_pairs_SAM.values(): 
        if pair[1]>0:
            true_det += 1   
        else:
            false_det += 1

    true_detected_cysts_SAM.append(true_det)
    false_detected_cysts_SAM.append(false_det)

    image_cystAn_name = os.path.join(dir_predicted, 'cystanalyser-'+base_name)
    image_cystAn = tif.imread(image_cystAn_name)
    image_cystAn_bin = 1*(image_cystAn>0) 

    ious_cystAn.append(compute_iou(image_GT_bin, image_cystAn_bin))
    numCysts_cystAn.append(len(np.unique(image_cystAn))-1)
    cystProp_cystAn.append(np.sum(image_cystAn_bin)/np.product(image_cystAn_bin.shape))
    matching_pairs_cystAn = check_matching_pairs(image_cystAn, image_GT)
    matching_pairs_means_cystAn.append(np.mean([pair[2] for pair in matching_pairs_cystAn.values()]))

    true_det = 0
    false_det = 0
    for pair in matching_pairs_cystAn.values(): 
        if pair[1]>0:
            true_det += 1
        else:
            false_det += 1

    true_detected_cysts_cystAn.append(true_det)
    false_detected_cysts_cystAn.append(false_det)
   


print('Mean +- std cyst proportion differences SAM: {} +- {}'.format(np.mean(np.abs(np.array(cystProp_GT)-np.array(cystProp_SAM))),\
                                                                     np.std(np.abs(np.array(cystProp_GT)-np.array(cystProp_SAM)))))  

print('Mean +- std cyst proportion differences cystAnalyser: {} +- {}'.format(np.mean(np.abs(np.array(cystProp_GT)-np.array(cystProp_cystAn))),\
                                                                              np.std(np.abs(np.array(cystProp_GT)-np.array(cystProp_cystAn)))))


print('Mean +- std proportion of true detected cysts over total detected cysts SAM: {} +- {}'.format\
      (np.mean(np.array(true_detected_cysts_SAM)/(np.array(true_detected_cysts_SAM)+np.array(false_detected_cysts_SAM))),\
       np.std(np.array(true_detected_cysts_SAM)/(np.array(true_detected_cysts_SAM)+np.array(false_detected_cysts_SAM)))))

print('Mean +- std proportion of true detected cysts over total detected cysts cystAnalyser: {} +- {}'.format\
      (np.mean(np.array(true_detected_cysts_cystAn)/(np.array(true_detected_cysts_cystAn)+np.array(false_detected_cysts_cystAn))),\
       np.std(np.array(true_detected_cysts_cystAn)/(np.array(true_detected_cysts_cystAn)+np.array(false_detected_cysts_cystAn)))))



Testing PKD2-10-1.svs - Series 2patch2544-3143_2080-2679.tif
Testing PKD2-10-1.svs - Series 2patch928-1527_2096-2695.tif
Testing PKD2-22-1.svs - Series 2patch4600-5199_2500-3099.tif
Testing PKD2-22-1.svs - Series 2patch5000-5599_4750-5349.tif
Testing PKD2-32-1.svs - Series 2patch2700-3299_3400-3999.tif
Testing PKD2-32-1.svs - Series 2patch9051-9650_2870-3469.tif
Testing PKD2-45-1.svs - Series 2patch3500-4099_5600-6199.tif
Testing PKD2-45-1.svs - Series 2patch8400-8999_4250-4849.tif
Testing PKD2-54-1.svs - Series 2patch4089-4688_4850-5449.tif
Testing PKD2-54-1.svs - Series 2patch5900-6499_3800-4399.tif
Testing PKD2-7-1.svs - Series 2patch3900-4499_3200-3799.tif
Testing PKD2-7-1.svs - Series 2patch987-1586_4542-5141.tif
Mean +- std cyst proportion differences SAM: 0.018642129629629633 +- 0.01931447298821311
Mean +- std cyst proportion differences cystAnalyser: 0.09242499999999998 +- 0.026211876809070913
Mean +- std proportion of true detected cysts over total detected cysts SAM: 0.589731